In [16]:
!pip install golem-node

In [17]:
!pip install golem-base-sdk python-dotenv

In [19]:
# Initialize the Golem DB client
from golem_base_sdk import GolemBaseClient
import json
import asyncio



from dotenv import load_dotenv
load_dotenv(".env")
import os

private_key_hex = os.getenv("PRIVATESS_KEY").replace("0x", "")# Using mock key for playground
print(private_key_hex)
# Convert hex to bytes


RPC_URL = os.getenv("RPC_URL", "https://ethwarsaw.holesky.golemdb.io/rpc")
WS_URL = os.getenv("WS_URL", "wss://ethwarsaw.holesky.golemdb.io/rpc/ws")


private_key = bytes.fromhex(private_key_hex)

# Create client connection
client = await GolemBaseClient.create_rw_client(
    rpc_url=RPC_URL,
    ws_url=WS_URL,
    private_key=private_key
)

# Get owner address
owner_address = client.get_account_address()
print(f"Connected with address: {owner_address}")
print("Ready to interact with Golem DB!")

3191e1d2b8e0bd58fd54ec1ce28c49ed20d04b0cc03e440515afc428e31c457c
Connected with address: 0xa34013043dfD69D1703377F75167b61b5a38AAF0
Ready to interact with Golem DB!


### Create entity

In [20]:
from golem_base_sdk import GolemBaseCreate, Annotation

async def create_first_entity(client):
    # Create entity
    entity = GolemBaseCreate(
        data=b"Hello, Golem DB from Python!",
        btl=100,  # Expires after 100 blocks
        string_annotations=[
            Annotation(key="type", value="greeting"),
            Annotation(key="language", value="python")
        ],
        numeric_annotations=[
            Annotation(key="priority", value=1),
            Annotation(key="version", value=3)
        ]
    )
    
    # Send to blockchain
    receipts = await client.create_entities([entity])
    receipt = receipts[0]
    
    print(f"Entity created!")
    print(f"Entity key: {receipt.entity_key}")
    print(f"Expires at block: {receipt.expiration_block}")
    
    return receipt.entity_key

### Query Entities

In [21]:
import json

async def query_entities(client, entity_key):
    print(f"Entity metadata: {await client.get_entity_metadata(entity_key)}")
    print(f"Entity storage: {await client.get_storage_value(entity_key)}")

    # 1. Simple equality query for strings (use double quotes)
    greetings = await client.query_entities('type="greeting"')
    print(f"Found {len(greetings)} greeting entities")

    # 2. Processing results
    for result in greetings:
        entity_key = result.entity_key
        decoded = result.storage_value.decode("utf-8")
        try:
            data = json.loads(decoded)
            print(f"Entity: {entity_key}, Decoded JSON data {data}")
        except (json.JSONDecodeError, ValueError):
            print(f"Entity: {entity_key}, Decoded data {decoded}")

    # 3. Numeric equality (no quotes for numbers)
    print(f"version_one: {await client.query_entities('version=1')}")
    print(f"high_priority: {await client.query_entities('priority=5')}")

    # 4. Numeric comparison operators
    print(f"above_threshold: {await client.query_entities('priority>3')}")
    print(f"old_versions: {await client.query_entities('version<10')}")
    print(f"in_range: {await client.query_entities('score>=80')}")

    # 5. Combining conditions with AND (&&)
    print(f"specific: {await client.query_entities('type="message" && version=2')}")
    print(f"filtered: {await client.query_entities('status="active" && priority>3')}")

    # 6. Using OR (||) for multiple options
    print(f"messages: {await client.query_entities('type="message" || type="alert"')}")

    # 7. Complex queries with parentheses
    print(
        f"complex_query: {
            await client.query_entities(
                '(type="greeting" && version>2) || status="urgent"'
            )
        }"
    )

    # 8. Query by owner with variable
    owner = client.get_account_address()
    print(f"by_owner: {await client.query_entities(f'$owner="{owner}"')}")

    # Note: String values need double quotes: type="message"
    # Numbers don't need quotes: priority=5
    # Use && for AND operator, || for OR operator in complex queries